In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('lending_club_ml.csv')
df.info()

C:\Users\leero\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (34,93) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2506738 entries, 0 to 2506737
Columns: 109 entries, id to length_of_cr
dtypes: float64(95), int64(4), object(10)
memory usage: 2.0+ GB


In [3]:
#df.fillna(0,inplace = True)

In [3]:
for thing in df:
    if df[thing].dtype == 'object':
        print(thing)    

grade
sub_grade
home_ownership
verification_status
purpose
initial_list_status
application_type
verification_status_joint
sec_app_earliest_cr_line
disbursement_method


In [5]:
df.application_type

0          Individual
1          Individual
2          Individual
3          Individual
4          Individual
5          Individual
6          Individual
7          Individual
8          Individual
9          Individual
10         Individual
11         Individual
12         Individual
13         Individual
14         Individual
15         Individual
16         Individual
17         Individual
18         Individual
19         Individual
20         Individual
21         Individual
22         Individual
23         Individual
24         Individual
25         Individual
26         Individual
27         Individual
28         Individual
29         Individual
              ...    
2506708    Individual
2506709    Individual
2506710    Individual
2506711    Individual
2506712    Individual
2506713     Joint App
2506714    Individual
2506715    Individual
2506716    Individual
2506717    Individual
2506718    Individual
2506719    Individual
2506720    Individual
2506721    Individual
2506722   

In [6]:
#pd.get_dummies(df)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,loan_status,dti,...,settlement_date_Oct-16,settlement_date_Oct-17,settlement_date_Oct-18,settlement_date_Sep-12,settlement_date_Sep-13,settlement_date_Sep-14,settlement_date_Sep-15,settlement_date_Sep-16,settlement_date_Sep-17,settlement_date_Sep-18
0,1077501,5000.0,5000.0,4975.00000,36,10.65,162.87,24000.00,1,27.65,...,0,0,0,0,0,0,0,0,0,0
1,1077430,2500.0,2500.0,2500.00000,60,15.27,59.83,30000.00,0,1.00,...,0,0,0,0,0,0,0,0,0,0
2,1077175,2400.0,2400.0,2400.00000,36,15.96,84.33,12252.00,1,8.72,...,0,0,0,0,0,0,0,0,0,0
3,1076863,10000.0,10000.0,10000.00000,36,13.49,339.31,49200.00,1,20.00,...,0,0,0,0,0,0,0,0,0,0
4,1075358,3000.0,3000.0,3000.00000,60,12.69,67.79,80000.00,1,17.94,...,0,0,0,0,0,0,0,0,0,0
5,1075269,5000.0,5000.0,5000.00000,36,7.90,156.46,36000.00,1,11.20,...,0,0,0,0,0,0,0,0,0,0
6,1069639,7000.0,7000.0,7000.00000,60,15.96,170.08,47004.00,1,23.51,...,0,0,0,0,0,0,0,0,0,0
7,1072053,3000.0,3000.0,3000.00000,36,18.64,109.43,48000.00,1,5.35,...,0,0,0,0,0,0,0,0,0,0
8,1071795,5600.0,5600.0,5600.00000,60,21.28,152.39,40000.00,0,5.55,...,0,0,0,0,0,0,0,0,0,0
9,1071570,5375.0,5375.0,5350.00000,60,12.69,121.45,15000.00,0,18.08,...,0,0,0,0,0,0,0,0,0,0
